We want to calculate the covariance matrix of our background data.



Fun fact: our covariance array is a Toeplitz matrix

In [125]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [126]:
full_time_series = torch.from_numpy(np.loadtxt("./data/background_0.dat", dtype=np.float32))
full_time_series = full_time_series.to(device)

time_series = full_time_series

In [156]:
torch.cuda.reset_peak_memory_stats()

cov_mat_row_one = []

sliding_window_size = 1000
batch_size = 100 # experimental so it runs well, must divide sliding_window_size evenly

y0_samples = time_series.unfold(0, sliding_window_size, 1).T[0]
y0_diff = y0_samples - torch.mean(y0_samples)

for i in range(int(sliding_window_size / batch_size)):
    samples = time_series.unfold(0, sliding_window_size, 1).T[i * batch_size:(i + 1) * batch_size]
    mean = torch.mean(samples)
    diff = samples - mean
    cov_mat_row_one.append(diff @ y0_diff / y0_samples.shape[0])
    del samples
    del mean
    del diff

# clear memory
with torch.no_grad(): # not sure if this line is correct
    torch.cuda.empty_cache()
    
cov_mat_row_one = torch.flatten(torch.stack(cov_mat_row_one))

print("Current MB allocated: " + str(torch.cuda.memory_allocated() / 1024 / 1024))
print("Max MB allocated: " + str(torch.cuda.max_memory_allocated() / 1024 / 1024))

Current MB allocated: 0.0
Max MB allocated: 0.0


In [157]:
torch.cuda.reset_peak_memory_stats()

cov_mat = torch.Tensor(scipy.linalg.toeplitz(cov_mat_row_one.cpu().numpy())).to(device)
inv_cov_mat = torch.inverse(cov_mat)

print("Current MB allocated: " + str(torch.cuda.memory_allocated() / 1024 / 1024))
print("Max MB allocated: " + str(torch.cuda.max_memory_allocated() / 1024 / 1024))

Current MB allocated: 0.0
Max MB allocated: 0.0


In [159]:
print(cov_mat)
print(inv_cov_mat)
print(cov_mat @ inv_cov_mat)

tensor([[21.2269,  0.2052,  0.5548,  ...,  0.1212,  0.1267,  0.1557],
        [ 0.2052, 21.2269,  0.2052,  ...,  0.1578,  0.1212,  0.1267],
        [ 0.5548,  0.2052, 21.2269,  ...,  0.1500,  0.1578,  0.1212],
        ...,
        [ 0.1212,  0.1578,  0.1500,  ..., 21.2269,  0.2052,  0.5548],
        [ 0.1267,  0.1212,  0.1578,  ...,  0.2052, 21.2269,  0.2052],
        [ 0.1557,  0.1267,  0.1212,  ...,  0.5548,  0.2052, 21.2269]])
tensor([[ 4.7925e-02,  3.0581e-04, -4.9541e-04,  ...,  2.7534e-05,
          1.4265e-05, -5.8326e-05],
        [ 3.0581e-04,  4.7927e-02,  3.0266e-04,  ..., -5.4640e-05,
          2.7022e-05,  1.4265e-05],
        [-4.9541e-04,  3.0266e-04,  4.7932e-02,  ..., -4.2612e-05,
         -5.4640e-05,  2.7534e-05],
        ...,
        [ 2.7534e-05, -5.4640e-05, -4.2612e-05,  ...,  4.7932e-02,
          3.0266e-04, -4.9541e-04],
        [ 1.4265e-05,  2.7022e-05, -5.4640e-05,  ...,  3.0266e-04,
          4.7927e-02,  3.0581e-04],
        [-5.8327e-05,  1.4265e-05,  2.